In [1]:
import pandas as pd
from pandas.io import gbq

In [4]:
get_track_listings = """
SELECT
  playlist_id,
  isrc,
  COUNT(DISTINCT pth.playlist_date) AS days_on_playlist
FROM
  `umg-partner.apple_music.playlist_track_history` pth
WHERE
  pth._PARTITIONTIME >= TIMESTAMP(DATE_SUB(CURRENT_DATE(), INTERVAL 367 DAY))
  AND pth._PARTITIONTIME < TIMESTAMP(DATE_SUB(CURRENT_DATE(), INTERVAL 2 DAY))
  AND playlist_id IN (
  SELECT
    playlist_uri
  FROM
    `umg-comm-tech-dev.recommender_playlists.all_playlists`
  GROUP BY
    playlist_uri)
GROUP BY
  playlist_id,
  isrc
"""

In [5]:
playlists_tracks = gbq.read_gbq(get_track_listings, project_id='umg-comm-tech-dev', dialect='standard')
playlists_tracks.shape

(1240701, 3)

In [7]:
gbq.to_gbq(playlists_tracks, destination_table='recommender_playlists.pt_apple',
           project_id='umg-comm-tech-dev', if_exists='replace')

1it [00:07,  7.31s/it]
